## Demo of Myria as a Federated Database


### Setup steps:
* Install Raco:
    * Follow the steps listed on https://github.com/uwescience/raco/tree/SPJA_federation#setup to set up RACO.
* Install SPARK:
    * Download the prebuilt HADOOP version from http://spark.apache.org/downloads.html and follow the setup steps on SPARK website to verify if it works. (Make sure you have all the dependencies listed on the page installed)
    * Set the SPARK_HOME environment variable to the downloaded SPARK folder and add it to PATH
        * On mac, this can be done by adding the following lines to bash_profile or bashrc file:
`
export SPARK_HOME=</path/to/spark>
export PATH=$SPARK_HOME/bin:$PATH
`

* Install jupyter:
    * on mac, this can be done via: `pip install jupyter`
    
* Run `ipython notebook` command. And browse to this notebook. 

* Update the path to the input dataset in the code below^: 
`
matA = scan('/path/to/datafile');
`

^ (As a sample, you can download and use https://github.com/uwescience/raco/blob/SPJA_federation/demo/sample.dat)


In [1]:
import time
import os
import raco.viz

import findspark
findspark.init()
findspark.add_packages('com.databricks:spark-csv_2.10:1.4.0')

from raco.backends.spark.connection import SparkConnection
from raco.backends.spark.catalog import SparkCatalog
from raco.backends.spark.algebra import SparkAlgebra

from raco.backends.logical import OptLogicalAlgebra
from raco.catalog import FromFileCatalog
import raco.myrial.interpreter as interpreter
import raco.myrial.parser as myrialparser

from raco.backends.myria import MyriaLeftDeepTreeAlgebra
from raco.backends.myria.connection import MyriaConnection
from raco.backends.myria.catalog import MyriaCatalog

from raco.backends.federated.connection import FederatedConnection
from raco.backends.federated.catalog import FederatedCatalog
from raco.backends.federated.algebra import FederatedAlgebra

In [2]:
masterHostname = os.environ.get('sparkurl', 'localhost')
def get_accumulo_connection():
    host='localhost'
    port=42424
    user='root'
    pw='secret'
    return AccumuloConnection(host, port, user, pw) 

def get_spark_connection():
    masterHostname = os.environ.get('sparkurl', 'localhost')
    if masterHostname == 'localhost':
        return SparkConnection(masterHostname)
    return SparkConnection("spark://{masterHostname}:7077".format(masterHostname=masterHostname))


In [3]:
conn = get_accumulo_connection()
conn.getTableProperties('public_adhoc_netflow')

NameError: global name 'AccumuloConnection' is not defined

In [3]:
##Initialize Backend 1
myriaconn = get_myria_connection()
myriacatalog = MyriaCatalog(myriaconn)

##Initialize Backend 2
sparkconn = get_spark_connection()
sparkcatalog = SparkCatalog.load_from_file(os.path.join(os.path.abspath('./examples/'), 'catalog.py'))

##Initialize Federated Backend
fed_conn = FederatedConnection([myriaconn, sparkconn])
fed_catalog = FederatedCatalog([myriacatalog, sparkcatalog])

In [4]:
##The Jaccard Computation expressed as MyriaL query
program_fquery="""
-- Scan netflow data (this resides in Accumulo)
NF = scan(netflow);
-- Filter the dataset (runs on Accumulo)
NFSUB = select SrcAddr as src_ip, DstAddr as dst_ip, 1.0 as value from NF where TotBytes > 100;
-- Scan DNS dataset (this is accessible to Spark)
DNS = scan('/home/dhutchis/gits/raco/examples/fed_accumulo_spark_c/dnssample_parsed.txt');
-- Perform a Join (Automatically move the datasets as necessary)
graph = select d1.dns as row, d2.dns as col, n.value
		from NFSUB n, DNS d1, DNS d2
	    where n.src_ip = d1.ip and n.dst_ip = d2.ip;

--Begin Jaccard
-- Calculate common neighbours
gammas = select a.row as u, b.row as v, count(b.value) as gamma
		 from graph a, graph b
         where a.col == b.col;

-- Calculate the out_degree of each vertex
out_d = select row, count(value) as od from graph;

-- Calculate the Jaccard Coefficients for all pairs (u,v);
J = select a.u as src_name, a.v as dst_name, a.gamma/(b.od + c.od - a.gamma) as jaccard_coeff
	from gammas a, out_d b, out_d c
    where a.u = b.row and a.v = c.row;

-- Store the result
store(J, nameJaccard);
"""

In [5]:
##Parse the query
parser = myrialparser.Parser()
processor = interpreter.StatementProcessor(fed_catalog, True)
statement_list = parser.parse(program_fquery)
processor.evaluate(statement_list)

In [6]:
##Lets Visualize the query
logical_plan = processor.get_logical_plan()
dot_logical = raco.viz.operator_to_dot_object(logical_plan)
dot_logical.view()

'Digraph.gv.pdf'

In [8]:
##Lets optimize the query and let the federation figure out the right backends
algebras = [OptLogicalAlgebra(), MyriaLeftDeepTreeAlgebra(), SparkAlgebra()]
falg = FederatedAlgebra(algebras, fed_catalog)
federated_plan = processor.get_physical_plan(target_alg=falg)

In [9]:
##Visualizing the physical plan
raco.viz.operator_to_dot_object(federated_plan).view()

'Digraph.gv.pdf'

In [ ]:
##Submit the plan for federated execution
fed_conn.execute_query(federated_plan)